In [1]:
# coding: utf-8
"""
simrankを用いた劣モジュラ最適化の検証
"""
from filer2.filer2 import Filer
from library.graphsubmodular import Partition_GraphSubModular
from library.rouge import Rouge_N
import numpy as np
import glob

In [2]:
start = 30
end = 40

list_test_path = glob.glob('./files/OpinosisDataset1.0/preprocessed3/*')
list_ans_path = glob.glob('./files/OpinosisDataset1.0/preprocessed1/ans/*')

list_test_path.sort()
list_ans_path.sort()

list_test_path = list_test_path[start:end]
list_ans_path = list_ans_path[start:end]

print len(list_test_path)
print len(list_ans_path)

10
10


In [ ]:
directed = False

counter = 1

for test_path, ans_path in zip(list_test_path, list_ans_path):
    dict_test = Filer.readdump(test_path)
    dict_ans = Filer.readdump(ans_path)
    filename = ans_path.replace('./files/OpinosisDataset1.0/preprocessed1/ans/', '')
    filename = filename.replace('.dump', '')
    print counter
    counter += 1
    # テスト用ファイルの作成
    # １００単語以上ある文は除く
    list_sentence = []
    list_sep = []
    list_sep_all = []
    list_pos = []
    list_edgelist = []
    for sen, sep, sepall, pos, edge in zip(dict_test['sentence'],
                                           dict_test['sep_all'],
                                           dict_test['sep_all'],
                                           dict_test['pos_all'],
                                           dict_test['edge_all']):
        if len(sepall) <= 100 and len(sep) != 0:
            list_sentence.append(sen)
            list_sep.append(sep)
            list_sep_all.append(sepall)
            list_pos.append(pos)
            list_edgelist.append(edge)

    list_edgelist = [[row1[0], row1[1]] for row in list_edgelist for row1 in row]
    # インスタンス化
    PGSM = Partition_GraphSubModular(list_sep_all=list_sep_all,
                                     list_sep=list_sep,
                                     list_pos=list_pos,
                                     list_edgelist=list_edgelist,
                                     directed=False,
                                     simrank_flag=True,
                                     weighted=False,
                                     log_flag=True)
    for n in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        n1 = float(n)/10
        for j in range(11-n):
            j1 = float(j)/10
            v = 10-n-j
            v1 = float(v)/10
            # 学習
            PGSM.m_greedy(num_w = 100,
                          r=1,
                          scale=0,
                          n=n1,
                          j=j1,
                          v=v1,
                          o=1.0/3)
            # Rougeを計算するようにbag_of_wordsを作成する
            list_C = PGSM.list_C
            list_gm_test = [list_sentence[row[0]].replace(',', '').replace('.', '').lower().split(' ') for row in list_C]
            list_gm_ans = [[sen.replace(',', '').replace('.', '').lower().split(' ') for sen in row] for row in dict_ans['sentence']]
            try:
                R1 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=1)
            except:
                R1 = 0
            try:
                R2 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=2)
            except:
                R2 = 0
            try:
                R3 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=3)
            except:
                R3 = 0
            try:
                R4 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=4)
            except:
                R4 = 0
            list_result = [[filename, 0, 1, n, v, j, R1, R2, R3, R4]]
            Filer.writecsv(list_result, './files/OpinosisDataset1.0/result/PGSM/all_rev2/all_%s_%s_%s.csv'%(n, j, v))